In [1]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\dg\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [2]:
from nltk.corpus import gutenberg
import pandas as pd

In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# DATA LOADING

In [4]:
with open('shakespeare-hamlet.txt','r') as f:
    text = f.read().lower()

In [5]:
tokenizer = Tokenizer()

In [6]:
tokenizer.fit_on_texts([text])

In [7]:
total_words = len(tokenizer.word_index)

In [8]:
total_words

4817

In [9]:
input_sequences = []

In [10]:
for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        

In [11]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

14

In [12]:
inputsequences = np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

In [13]:
import tensorflow as tf

In [14]:
x,y = inputsequences[:,:-1], inputsequences[:,-1]

In [15]:
y = tf.keras.utils.to_categorical(y,num_classes=total_words)

In [16]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU

## MODEL CREATION

In [56]:
model = Sequential()

In [57]:
model.add(Embedding(total_words+1,100,input_length=max_sequence_len-1))

In [58]:
model.add(LSTM(150,return_sequences=True))

In [59]:
model.add(Dropout(0.2))

In [60]:
model.add(LSTM(100))

In [61]:
model.add(Dense(total_words,activation='softmax'))

In [62]:
history = model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# MODEL TRAINING

In [63]:
from tensorflow.keras.callbacks import EarlyStopping

In [64]:
earlystopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [70]:
model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1)

Epoch 1/100
644/644 [==============================] - 23s 36ms/step - loss: 6.4384 - accuracy: 0.0375 - val_loss: 6.9585 - val_accuracy: 0.0441
Epoch 2/100
644/644 [==============================] - 21s 32ms/step - loss: 6.2437 - accuracy: 0.0476 - val_loss: 7.0450 - val_accuracy: 0.0503
Epoch 3/100
644/644 [==============================] - 20s 31ms/step - loss: 6.1086 - accuracy: 0.0505 - val_loss: 7.0337 - val_accuracy: 0.0513
Epoch 4/100
644/644 [==============================] - 20s 31ms/step - loss: 5.9864 - accuracy: 0.0526 - val_loss: 7.0882 - val_accuracy: 0.0532
Epoch 5/100
644/644 [==============================] - 20s 30ms/step - loss: 5.8656 - accuracy: 0.0602 - val_loss: 7.1417 - val_accuracy: 0.0612
Epoch 6/100
644/644 [==============================] - 19s 30ms/step - loss: 5.7425 - accuracy: 0.0659 - val_loss: 7.2017 - val_accuracy: 0.0569
Epoch 7/100
644/644 [==============================] - 20s 32ms/step - loss: 5.6142 - accuracy: 0.0723 - val_loss: 7.2881 - val_ac

644/644 [==============================] - 21s 32ms/step - loss: 2.4105 - accuracy: 0.4688 - val_loss: 11.9013 - val_accuracy: 0.0528
Epoch 58/100
644/644 [==============================] - 21s 32ms/step - loss: 2.3841 - accuracy: 0.4712 - val_loss: 11.9810 - val_accuracy: 0.0503
Epoch 59/100
644/644 [==============================] - 20s 31ms/step - loss: 2.3534 - accuracy: 0.4785 - val_loss: 12.0290 - val_accuracy: 0.0513
Epoch 60/100
644/644 [==============================] - 20s 31ms/step - loss: 2.3349 - accuracy: 0.4827 - val_loss: 12.0975 - val_accuracy: 0.0480
Epoch 61/100
644/644 [==============================] - 21s 32ms/step - loss: 2.2995 - accuracy: 0.4908 - val_loss: 12.1957 - val_accuracy: 0.0468
Epoch 62/100
644/644 [==============================] - 20s 32ms/step - loss: 2.2767 - accuracy: 0.4953 - val_loss: 12.2246 - val_accuracy: 0.0486
Epoch 63/100
644/644 [==============================] - 21s 33ms/step - loss: 2.2533 - accuracy: 0.4981 - val_loss: 12.2963 - val_a

# PREDICTIONS

In [71]:
def predict_next_word(model,tokenizer,text,max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]
    token_list = pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict(token_list,verbose=1)
    predicted_word_index = np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None

In [72]:
text = "Actus Primus. Scoena Prima.Enter Barnardo and Francisco"
print(predict_next_word(model,tokenizer,text,max_sequence_len))

1/1 [==============================] - 0s 40ms/step
and


In [75]:
text = "Fran. For this releefe much thankes: 'Tis"
print(predict_next_word(model,tokenizer,text,max_sequence_len))

1/1 [==============================] - 0s 35ms/step
bitter


In [74]:
text = "Barn. Well, goodnight. If you do meet Horatio"
print(predict_next_word(model,tokenizer,text,max_sequence_len))

1/1 [==============================] - 0s 29ms/step
and
